In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Input, Dropout
from keras.layers import Dense

In [4]:
import pandas as pd
data = pd.read_pickle("data/modelos.pkl")
data['modelo']=data['modelo'].apply(lambda x: x.split('/')[0])
data.shape
data.head(5)

,star_rating,sentiment,score,review_body,modelo,star_mapped
0,5,0,0.999859,Brighter then OEM,mrm8488,1
0,5,0,0.999871,Works every time using apps on my Windows Lumi...,mrm8488,1
0,5,0,0.966300,Very cool and helpful.,mrm8488,1
0,5,0,0.999843,I've had them in for about a month now and I b...,mrm8488,1
0,4,-1,0.997117,Bluetooth connection can lag at times but over...,mrm8488,1


In [5]:
df = pd.read_pickle("data/modelos_ensemble.pkl")
print(df.shape)
df.drop(columns=['index'],inplace=True)
print(df)

(160665, 7)
        target  s1  s2  s3  s4  s5
0            1   0   0   0   0   1
1            1   0   0   0   1   0
2            1   0   0   0   1   0
3            1   0   0   0  -1   0
4            1  -1   1   1   1   1
...        ...  ..  ..  ..  ..  ..
160660       1   1   1   1   1   1
160661       1   1   0   0   1   1
160662      -1   0   0   0  -1  -1
160663       1   0   0   0   1  -1
160664       1   0   1   0   1   1

[160665 rows x 6 columns]


In [6]:
from sklearn.model_selection import train_test_split

train=df.iloc[:,1:6]
y=df['target']
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)

In [7]:
y_train_tf =  tf.keras.utils.to_categorical(y_train, 3)
y_test_tf =  tf.keras.utils.to_categorical(y_test, 3)
print(y_train_tf,y_train)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]] 105733    1
53378     1
80497    -1
70443     1
55675     1
         ..
119879    1
103694    1
131932    1
146867    1
121958    1
Name: target, Length: 128532, dtype: int8


In [8]:
from sklearn.utils import class_weight
import numpy as np
import math

def create_class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weight = dict()
    
    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0
    
    return class_weight

# random labels_dict
labels_dict = {0:len(y_train[y_train==-1]),1: len(y_train[y_train==0]),2:len(y_train[y_train==1])}

class_weight=create_class_weight(labels_dict)

In [9]:
#This callback will stop the training when there is no improvement in the loss for the consecutive nº of epochs defined in patience parameter.
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define model
t_model = Sequential()
t_model.add(Dense(25, input_dim=5, activation='relu'))
t_model.add(Dense(3, activation='softmax'))
t_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
t_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                150       
                                                                 
 dense_1 (Dense)             (None, 3)                 78        
                                                                 
Total params: 228
Trainable params: 228
Non-trainable params: 0
_________________________________________________________________


In [10]:
# fit model
history = t_model.fit(X_train, y_train_tf, validation_data=(X_test, y_test_tf), epochs=10, callbacks=[callback],class_weight=class_weight, verbose=1)
t_model.save('saved_model/t_model')

Epoch 1/10
4017/4017 [==============================] - 22s 5ms/step - loss: 0.3252 - accuracy: 0.9077 - val_loss: 0.2974 - val_accuracy: 0.9083
Epoch 2/10
4017/4017 [==============================] - 17s 4ms/step - loss: 0.3065 - accuracy: 0.9103 - val_loss: 0.2965 - val_accuracy: 0.9088
Epoch 3/10
4017/4017 [==============================] - 17s 4ms/step - loss: 0.3060 - accuracy: 0.9103 - val_loss: 0.2957 - val_accuracy: 0.9088
Epoch 4/10
4017/4017 [==============================] - 18s 5ms/step - loss: 0.3057 - accuracy: 0.9103 - val_loss: 0.2963 - val_accuracy: 0.9087
Epoch 5/10
4017/4017 [==============================] - 17s 4ms/step - loss: 0.3054 - accuracy: 0.9101 - val_loss: 0.2950 - val_accuracy: 0.9088
Epoch 6/10
4017/4017 [==============================] - 16s 4ms/step - loss: 0.3050 - accuracy: 0.9102 - val_loss: 0.2958 - val_accuracy: 0.9083
Epoch 7/10
4017/4017 [==============================] - 17s 4ms/step - loss: 0.3047 - accuracy: 0.9103 - val_loss: 0.2939 - val_ac

In [11]:
t_model = tf.keras.models.load_model('saved_model/t_model')
# Check its architecture
t_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                150       
                                                                 
 dense_1 (Dense)             (None, 3)                 78        
                                                                 
Total params: 228
Trainable params: 228
Non-trainable params: 0
_________________________________________________________________


In [12]:
t_model_pred = t_model.predict(X_test)
print(t_model_pred)

1005/1005 [==============================] - 4s 4ms/step
[[0.01040052 0.98811007 0.00148937]
 [0.01200145 0.97871315 0.00928547]
 [0.01499161 0.9786689  0.00633939]
 ...
 [0.1416324  0.24288434 0.61548334]
 [0.01040052 0.98811007 0.00148937]
 [0.00830026 0.98501205 0.00668771]]


In [13]:
import numpy as np

t_model_predictions = np.argmax(t_model_pred, axis=1) # select the index with the highest probability

fx = (lambda x: {0:0, 1:1, 2:-1}[x])

t_model_predictions = np.array([fx(x) for x in t_model_predictions]) # map the index to the corresponding label

In [14]:
from sklearn.metrics import classification_report
print(classification_report(y_test, t_model_predictions))

              precision    recall  f1-score   support

          -1       0.63      0.72      0.67      2738
           0       0.45      0.01      0.01      1441
           1       0.94      0.97      0.96     27954

    accuracy                           0.91     32133
   macro avg       0.67      0.57      0.55     32133
weighted avg       0.89      0.91      0.89     32133

